## Seminar 10. Retrieval‑Augmented Generation (RAG)

In this seminar we will make an LLM assistant that can answer arbitrary questions about films in Russian. For this we will use RAG and a dataset with reviews from Kinopoisk.

### Agenda

1. How does the model perform without RAG?
1. Building a RAG on feedback
1. Reranker
1. Multi-Query
1. Filtering by meta-information

In [25]:
import numpy as np
import uuid
from tqdm.auto import tqdm

### Dataset and model

We will download our [dataset](https://huggingface.co/datasets/blinoff/kinopoisk) from HugingFace Hub. It contains over 35.000 user reviews of various films.

In [45]:
from datasets import load_dataset

def process_dataset(sample):
    sample['content'] = sample['content'].replace('\xa0', ' ')
    return sample

dataset = load_dataset("blinoff/kinopoisk")['train']
dataset = dataset.map(process_dataset)

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

kinopoisk.jsonl:   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36591 [00:00<?, ? examples/s]

Map:   0%|          | 0/36591 [00:00<?, ? examples/s]

In [46]:
dataset

Dataset({
    features: ['part', 'movie_name', 'review_id', 'author', 'date', 'title', 'grade3', 'grade10', 'content'],
    num_rows: 36591
})

In [47]:
dataset[0]['movie_name']

'Блеф (1976)'

In [48]:
dataset[0]['content']

'\n"Блеф» — одна из моих самых любимых комедий.\n\nЭтот фильм я наверно смотрел раз сто, нет я конечно блефую, я видел его куда больше. Не могу не выразить своё восхищение главными действующими лицами этого фильма. Начну с Адриано Челентано для которого как я считаю это лучшая роль в кино. Великолепный актёр, неплохой певец, странно что на его родине в Италии его песни мало кто слушает. Ну я думаю что и итальянцы и французы привыкли к тому, что у нас до сих их актёры популярней чем даже на своей родине. Да, такой вот парадокс. Челентано конечно профессионал своего дела, комик с серьёзным выражением лица. Он смешон ещё и потому, что одновременно так серъёзен. Адриано браво!\n\nА теперь несколько слов об Энтони Куине. Да тот самый горбун из Нотр-дама. Собор Парижской Богоматери, оригинальная версия, кто не смотрел рекомендую. С ним как-то приключилась одна интересная история. На съёмках одного из своих фильмов он то ли сломал, то ли подвихнул ногу, а роль требовала от него чтобы в одной 

We will perform all experiments with the model [`Qwen/Qwen2-1.5B-Instruct`](https://huggingface.co/Qwen/Qwen2-1.5B-Instruct). This is a GPT-based question-answer model that has been trained in a large number of languages, including Russian.

In [3]:
from transformers import pipeline
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

generation_pipeline = pipeline(
    "text-generation",
    model="Qwen/Qwen2-1.5B-Instruct",
    device=device,
    torch_dtype=torch.float16
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Generation without RAG

Let's check how well the model answers the questions without using RAG.

In [291]:
query = 'В каких пяти фильмах играл Роберт де Ниро?'

In [260]:
messages = [
    {"role": "user", "content": query},
]
output = generation_pipeline(messages, max_new_tokens=256, do_sample=True, temperature=0.9, top_p=0.7)

answer = output[0]['generated_text'][1]['content']

print(answer)

Роберт Де Ниро сыграл в следующих пяти фильмах:

1. "Богемская революция" (1968)
2. "Терминатор" (1984)
3. "Леон: Охотник на дичь" (1994)
4. "Американский квадрип" (1995)
5. "Суони" (2019)


We see that the knowledge of the model is lacking. All names, except Terminator, refer to non-existent films, and Robert De Niro did not play in Terminator.

##  Retrieval‑Augmented Generation

Let's try to improve the model quality using RAG. To do this, we first need to compose a vector database. As an embedding model we will use [`intfloat/multilingual-e5-large`](https://huggingface.co/intfloat/multilingual-e5-large). This is a large multilingual model based on Transformer's Encoder.

In [263]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("intfloat/multilingual-e5-large", model_kwargs={'torch_dtype': torch.float16})

We will build our database based on the tool [`Qdrant`](https://github.com/qdrant/qdrant-client). It implements various methods for text searching, so we don't have to write anything by hand. It is enough to provide it with text embeddings.

In [ ]:
from qdrant_client import QdrantClient, models

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="kinopoisk_e5",
    on_disk_payload=True,
    vectors_config=models.VectorParams(
        size=1024,
        distance=models.Distance.COSINE,
        on_disk=True
    ),
)

To split text into chunks, we use `RecursiveCharacterTextSplitter` from the [`langchain`](https://github.com/langchain-ai/langchain) library. We will recursively split the text into chunks of about 1000 characters.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

Assembling a vector database.

In [ ]:
for i in tqdm(range(len(dataset))):
    text_chunks = text_splitter.split_text(dataset[i]['content'])

    vectors = embedding_model.encode(text_chunks, normalize_embeddings=True, device=device).tolist()

    client.upsert(
        collection_name='kinopoisk_e5',
        points=[
            models.PointStruct(
                id=str(uuid.uuid4()),
                vector=vectors[j],
                payload={
                    'text': text_chunks[j],
                    'movie_name': dataset[i]['movie_name'][:-7],
                    'year': int(dataset[i]['movie_name'][-5:-1]),
                }
            )
            for j in range(len(text_chunks))
        ]
    )

Let's check how well we can find similar texts.

In [ ]:
query_vector = embedding_model.encode(query, normalize_embeddings=True, device=device).tolist()

In [215]:
hits = client.search(
    collection_name="kinopoisk_e5",
    query_vector=query_vector,
    limit=5
)

In [216]:
[hit.payload for hit in hits]

[{'text': 'Хороший фильм. На реальных событиях. Игра де Ниро восхищает. Робин Уильямс как обычно смешной и трагический.',
  'movie_name': 'Пробуждение',
  'year': 1990},
 {'text': 'Потрясающий фильм. Великолепная игра Роберта Де Ниро. Изумительная по красоте и, как нельзя подходящая к этой кинокартине, музыка Эннио Морриконе. Этот шедевр стоит того, чтобы смотреть и смотреть не один раз.',
  'movie_name': 'Однажды в Америке',
  'year': 1983},
 {'text': 'После долгих и продуктивных лет работы в качестве актера, Роберт де Ниро решил попробовать себя и в режиссерском поприще. Первый фильм этого гениального человека собрал в Америке больше 17 миллионов долларов. Интригующее повествование о судьбе обычного молодого парня, которому предстоит сделать самый важный шаг в своей жизни и, тем самым выбрать свой дальнейший путь существования, пришлось зрителю по душе. Семейная драма то и дело перекрикивается с гангстерским эпосом, и это делает фильм просто уникальным. Добавьте сюда щепотку романтик

Great! All the retrieved texts are related to Robert De Niro. Now let's wrap this search process in a function.

In [242]:
def semantic_search(client, query, limit=10):
    query_vector = embedding_model.encode(
        query, normalize_embeddings=True, device=device
    ).tolist()

    hits = client.search(
        collection_name="kinopoisk_e5",
        query_vector=query_vector,
        limit=limit
    )
    relevant_chunks = [hit.payload for hit in hits]

    return relevant_chunks

### RAG on reviews

To implement RAG, we will pass the feedback relevant to the query into the context of the model.

In [264]:
def llm_answer(query, context):
    prompt = f"""
    Ты русскоязычный эксперт в области кинематографа. У тебя есть доступ к набору отзывов о фильмах, используй их, чтобы полно и точно ответить на следующий вопрос. Убедись, что ответ подробный, конкретный и непосредственно касается вопроса. Не добавляй информацию, которая не подтверждается предоставленными отзывами.

Вопрос:
{query}

Отзывы:
{context}
"""
    messages = [
        {"role": "user", "content": prompt},
    ]
    output = generation_pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.9, top_p=0.7)

    return output[0]['generated_text'][1]['content']

In [274]:
def predict(query):
    selected_chunks = semantic_search(client, query)
    context = ' ; '.join([f"Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context)

In [276]:
print(predict(query))

Роберт де Ниро играл в следующих пяти фильмах:

1. "Загадочный человек" (The Departed)
2. "Огонь" (Heat)
3. "Тайны семьи" (The Departed)
4. "Алиса в стране чудес" (Alice in Wonderland)
5. "Мстители" (Captain America: Civil War)


Despite the added context, the results aren't good. The thing is that reviews very rarely contain the movie titles themselves. Let's try to add titles to the context as well.

### Adding movie titles

In [268]:
def predict(query):
    selected_chunks = semantic_search(client, query)
    context = ' ; '.join([f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context)

In [269]:
print(predict(query))

Роберт де Ниро играл в следующих пяти фильмах:

1. Пробуждение (2006) - отзыв: "Хороший фильм. На реальных событиях. Игра де Ниро восхищает. Робин Уильямс как обычно смешной и трагический."
2. Однажды в Америке (1999) - отзыв: "Уникальный фильм. Игра Роберта Де Ниро не может не вызвать восхищения. Он действительно великолепен."
3. Пробуждение (2006) - отзыв: "Фильм о пробуждении души того человека, который пробудил физически больных 'хроников'."
4. Бронкская история (2005) - отзыв: "Богатейший актерский опыт Роберта Де Ниро с такими авторитетными режиссерами криминальных драм как Мартин Скорцезе, Брайан Де Пальма, Френсис Форд Коппола просто не мог пройти даром. И талантливый актер сделал не менее изящный, подобно десятку своих ролей, режиссерский дебют картиной 'Бронкская история'."
5. Схватка (2001) - отзыв: "Отдельно стоит упомянуть два эпизода фильма — перестрелку на улице и финальную разборку Аль Пачино с Де Ниро. Это действительно круто. Как в самом крутом боевике."


Now the model prints all the movies that Robert De Niro actually acted in.

## Reranker

Let's try to improve the method by adding reranking.

<img src="https://2.downloader.disk.yandex.ru/preview/562f6ca63a4c9c167c7104adcb69038c62397d1ff7dd746ef264d65aa149d797/inf/WyZ1fToxI4Te6PT0CXKY6FgaekwIOFx7YP6UyIKPqW4IbXlkBZs3XoE4BLVVAGicFSVd-q4LT1GZtOApSPvsUA%3D%3D?uid=676720824&filename=reranker.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=676720824&tknv=v2&size=3024x1688" alt="drawing" width="700"/>

Reranker is a language model that takes two texts and returns the proximity between them.

<img src="https://4.downloader.disk.yandex.ru/preview/33e900424814cb2806c8f6a00cd39642d4ef85610beb7eb8b0b3c69aedac59c5/inf/wrfaZRuiUUopUMh223NnlEQlalUZHKqP6BDrYSkm2KX9gJHGkcE-HKvmT6z42V_6s6WvDU_hpBb67l0nllK5Ng%3D%3D?uid=676720824&filename=reranker_arch.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=676720824&tknv=v2&size=3024x1688" alt="drawing" width="200"/>

As a reranker, we will use a special multi-language model trained for this purpose [`amberoad/bert-multilingual-passage-reranking-msmarco`](https://huggingface.co/amberoad/bert-multilingual-passage-reranking-msmarco).

In [277]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

In [278]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='amberoad/bert-multilingual-passage-reranking-msmarco',
    model_kwargs={'device': 'cpu'}
)
sum([p.numel() for p in cross_encoder.client.model.parameters()])

167357954

In [304]:
def predict(query):
    selected_chunks = semantic_search(client, query, limit=50)

    texts = [f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks]
    scores = cross_encoder.score([(query, text) for text in texts])

    idxs = np.argsort(list(scores))[-10:]

    context = ' ; '.join([texts[i] for i in idxs])
    return llm_answer(query, context), context

In [305]:
answer, context = predict(query)

print(answer)

В фильмах, в которых играл Роберт де Ниро:

1. Бронкская история
2. Однажды в Америке 
3. Пробуждение
4. Военный ныряльщик
5. Крестный отец 2


It did get a little better.

In [306]:
context.split(' ; ')

['Название: Бронкская история. Отзыв: Богатейший актерский опыт Роберта Де Ниро с такими авторитетными режиссерами криминальных драм как Мартин Скорцезе, Брайан Де Пальма, Френсис Форд Коппола просто не мог пройти даром. И талантливый актер сделал не менее изящный, подобно десятку своих ролей, режиссерский дебют картиной «Бронкская история».\n\nСыграв также одну из второстепенных ролей, Роберт Де Ниро подарил необычный ему образ правильного и положительного героя, который из всех сил пытается вырастить в своем сыне мужество и справедливость. Ровно, как и герой Чазза Пальминтери, «крестного отца» итальянского квартала, которого, собственно больше боятся, чем любят. \n\nНо и герой Чазза Пальминтери по-своему прав, взяв на воспитание сына Лоренцо, он не утаивает сложность своего места под солнцем, и учит молодого парня в двух направлениях — школы и улицы одновременно.',
 'Название: Однажды в Америке. Отзыв: Естественно вы из всего вышеперечисленного не поймете, почему этот фильм настолько

Let's try some other questions.

In [312]:
query = 'Какой фильм выиграл больше всего оскаров?'
answer, context = predict(query)

print(answer)

Фильм "Титаник" выиграл наибольшее количество Оскаров - 11.


In [313]:
context.split(' ; ')

['Название: Амадей. Отзыв: Шикарные декорации, потрясающая музыка, отличная актерская игра, прекрасный сценарий оставляют просто великолепные ощущения после просмотра данного киношедевра. Да и 8 «Оскаров» и 3 номинации говорят сами за себя. \n\nОдин из наиболее успешных фильмов в истории Киноакадемии. Браво Милош Форман, браво актеры, браво Моцарт!',
 'Название: Король говорит!. Отзыв: После абсолютно немотивированных главных победителей «Оскара» последних двух лет — «Миллионера из трущоб» и «Повелителя бури» — объявления победителя 2011 года я ждал с изрядной долей опасения. К счастью, опасения оказались напрасными: посмотрев позднее фильм «Король говорит!» я, по крайней мере, могу понять логику рассуждений американских киноакадемиков и согласиться что лента об английском монархе — фильм достойный и стал лучшим по праву. Хотя, признаюсь, я больше бы обрадовался триумфу умного, психологически тонкого «Начала» Кристофера Нолана или простой, но вместе с тем гениальной в своей простоте «Ж

Here the model answered "Titanic". Among all movies in context, this one had the most amount of Oscars.

Thanks to prompt, the model does not answer a question if the context does not have the right information!

In [232]:
query = 'Сколько лет Тому Холланду?'
answer, context = predict(query)

In [233]:
answer

'Извините, но я не могу предоставить вам информацию об возрасте Тома Холланда, так как все отзывы, которые вы указали, относятся к другим фильмам и не содержат информации о его возрасте.'

In [234]:
context.split(' ; ')

['Название: Гран Торино, отзыв: Спросите у себя — сколько вы знаете режиссеров, способных снимать по два фильма в год? Сколько вы знаете режиссеров, способных снимать по два фильма в год, которые с легкостью претендуют в «топ-20 года». Сколько должно быть лет такому режиссеру? Клинту Иствуду уже семьдесят восемь, но его работоспособности позавидует любой начинающий режиссер, готовый трудиться годами без сна.',
 'Название: Начало, отзыв: И на закуску из мужчин я оставила Тома Харди. Тут ооо и только ооо. Потом я очнулась и стала следить за тем, как он играет. Вообще, появившись в костюме какого-то местного клоуняки или назовём этот образ — первый парень на селе — я недоверчиво покрутила головой. Потом гляжу — мой мальчик таки выбился в большое кино к 32 годам. Успех. Отменно отыграл и причём реально большую роль, показав, что способен быть и самостоятельным экшн-героем. Я у экрана просто сходила с ума. Хорошо, что хоть выла тихонько, а не в полную силу.\n\nБарышни тоже не подкачали. Пей

## Multi-Query

Let's try to broaden the context by adding paraphrased questions.

<img src="https://4.downloader.disk.yandex.ru/preview/01a5d4071ab3986558f902e40f948e6f09340eb403f5dcfebf538dc160c02876/inf/XjsYIV3dwpNS9wBGWwcq8TGxcV_7jDFT8GesvBQhzWOil1U3gBaVHRdEfVM6JUOFdc7htSKOy2Na7VPYMjW95g%3D%3D?uid=676720824&filename=multi-query.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=676720824&tknv=v2&size=3024x1688" alt="drawing" width="550"/>

For paraphrasing, we will use the same model we use to generate the text, but with a new prompt.

In [1]:
import re

def rephrase_query(query, n=3):
    prompt = f"""
Твоя задача написать {n} разных вариаций вопроса пользователя для того,
чтобы по ним получить релевантные документы из векторной базы данных.
Ты должен переформулировать вопрос с разных точек зрения.
Это поможет избавить пользователя от недостатков поиска похожих документов на основе расстояния.
Вопрос пользователя сфокусирован на теме кино.
Напиши ТОЛЬКО вариации вопроса и больше ничего, разделяя их символом новой строки \\n.
НЕ пиши ответ на сам вопрос.
-----------------
{query}

"""
    messages = [
        {"role": "user", "content": prompt},
    ]
    output = generation_pipeline(messages, max_new_tokens=512, do_sample=True, temperature=0.9, top_p=0.7)
    queries = output[0]['generated_text'][1]['content']

    return re.split(r'\n+', queries)

def predict(query):
    queries = rephrase_query(query, n=3)

    all_chunks = []
    for rephrased_query in queries:
        selected_chunks = semantic_search(client, rephrased_query, limit=5)
        all_chunks.extend(selected_chunks)

    context = [f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in all_chunks]

    context = ' ; '.join(np.unique(context))

    answer = llm_answer(query, context)

    return answer, context, queries

In [321]:
query = 'Посоветуй легкую комедию'
answer, context, queries = predict(query)

In [322]:
print(answer)

На основе представленных отзывов и их обобщения, могу предложить следующие легкие и комедийные фильмы:

1. "Любовь и голуби". Этот фильм, написанный автором Лайонелом Купером, часто называют лучшей работой этого режиссера. Он известен своим уникальным юмором и легкостью в рассказе истории.

2. "Бриллиантовая рука". Этот фильм стал культовым благодаря своей неожиданной исторической реальностью и оригинальным персонажам.

3. "Пролетая над гнездом кукушки". Это фильм-триллер с увлекательной историей, где актеры показывают себя в самых разных профессиях.

4. "Рататуй". Этот фильм - это настоящая шутка, и его создатели стремятся удивить вас всеми возможными способами.

5. "Семь жизней". Этот фильм, как правило, считается наиболее серьезным и серьезным в репертуаре кинематографиста Эмилии Грант.

Однако, стоит отметить, что многие люди могут найти фильм "Обитаемый остров" очень веселым и забавным, особенно если они предпочитают комедию.

Также важно помнить, что каждый человек имеет свои пре

The quality of Multi-Query depends a lot on the LLM's ability to paraphrase the text. If it does this poorly, irrelevant queries will appear. This can be fixed by adding filtering on the relevance of the input query.

In [323]:
queries

['Попробуйте рассмотреть следующие варианты вопроса:',
 '1) "Существуют ли любые фильмы-комедии, которые можно описать как легкие?"',
 '2) "Чем могу заняться, чтобы найти фильм-комедию без особого труда?"',
 '3) "Какие из популярных фильмов-комедий считаются наиболее легкими?"']

In [324]:
context.split(' ; ')

['Название: Бойцовский клуб. Отзыв: пересматривая который можно открыть что-то новое и каждый раз задумываясь о просмотренном?Или это всего лишь фильм на определенное время, удачно подвернувшийся когда надо?',
 'Название: Братц. Отзыв: Советую посмотреть. \n\n\n\n6 из 10',
 'Название: Бриллиантовая рука. Отзыв: Что можно еще добавить? Разве только то, что когда слышишь название этого фильма, или одну из множества цитат, на которые буквально порвали ленту — сразу же начинаешь улыбаться. А на душе становится легко и весело. \n\nТак нужны ли иные критерии оценки фильма?',
 'Название: В джазе только девушки. Отзыв: Очаровательный фильм! Такой и должна быть классическая комедия: интересной, игривой и легкой, как бизе!',
 'Название: В джазе только девушки. Отзыв: любят погорячее») отличается, лёгким и беззаботным юмором, этот фильм можно смотреть в любом возрасте, в любом настроении, совершенно разным людям.',
 'Название: Любовь и голуби. Отзыв: И самое главное герои фильма отвечают на эти в

## Filters

For some types of queries, you can add filters to prevent documents that do not fit into the context. For example, we want to find out something about movies from the particular year. If such information is not in the review itself, we can't utilize it in a semantic search.

In [345]:
def predict(query):
    selected_chunks = semantic_search(client, query, limit=10)
    context = ' ; '.join([
        f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}"
        for chunk in selected_chunks])

    return llm_answer(query, context), context

In [364]:
query = 'Составь список пяти лучших мелодрам 1980-х годов'
answer, context = predict(query)

In [365]:
print(answer)

Список пяти лучших мелодрам 1980-х годов:

1. "Привидение" (The Ghosts of Christmas Past, Present and Future) - Этот фильм, написанный Ричардом Лоренсом, стал классикой 80-х. Он имеет простую сюжетку о двух младших братах, которые пытаются открыть секреты своего отца, которого они знают лишь по его фотографиям.

2. "Крамер против Крамера" (Cramer vs Kramer) - Это комедийный фильм, основанный на реальных событиях. Фильм рассказывает историю молодого мужчины, который вынужден разделить обязанности с матерью своей дочери, которая не хочет видеть его после развода.

3. "Клуб 'Завтрак'" (Club Med) - Этот фильм является комедией с элементами драмы. Он рассказывает о группе друзей, которые вместе проводят выходные в клубе "Завтрак".

4. "Дневник памяти" (Memoirs of a Geisha) - Этот фильм представляет собой историю о девочки, которая живет в Японии и становится проституткой. Фильм также показывает ее отношения с другим девушкой.

5. "Звездный десант 3 Мародер" (Starship Troopers 3: Marauder) -

В этом примере модель сгенерировала не только не мелодрамы, но и фильмы неверных лет. Заметьте, что в годах присутствуют галлюцинации.

In [366]:
def filtered_semantic_search(client, query, filter_years, limit=10):
    query_vector = embedding_model.encode(
        query, normalize_embeddings=True, device=device
    ).tolist()

    begin, end = filter_years
    hits = client.search(
        collection_name="kinopoisk_e5",
        query_vector=query_vector,
        limit=limit,
        query_filter=models.Filter(
            must=[models.FieldCondition(key="year", range=models.Range(gte=begin, lte=end))]
        ),
    )
    relevant_chunks = [hit.payload for hit in hits]

    return relevant_chunks

def predict(query, filter_years=None):
    if filter_years is not None:
        selected_chunks = filtered_semantic_search(client, query, filter_years=filter_years, limit=10)
    else:
        selected_chunks = semantic_search(client, query, limit=10)

    context = ' ; '.join([f"Название: {chunk['movie_name']}. Отзыв: {chunk['text']}" for chunk in selected_chunks])

    return llm_answer(query, context), context

In [367]:
answer, context = predict(query, filter_years=(1980, 1989))

In [368]:
print(answer)

Список пяти лучших мелодрам 1980-х годов:

1. "Клуб 'Завтрак'"
2. "Ганди"
3. "Назад в будущее 2" 
4. "Назад в будущее"
5. "Однажды в Америке"

Эти фильмы отличаются своими уникальными сюжетами, великолепными актерскими играми, драматическими моментами и замечательным саундтреком. Каждый из них имеет свою уникальную историю, которая может вызвать волну эмоций у зрителя.


Note that although all the films got the right year, almost all of them aren't melodramas.